# Import lib

In [1]:
import pickle
import numpy as np
import string
import re
from pyvi import ViTokenizer
import matplotlib.pyplot as plt
import nltk

from tkinter import *
#from nltk.tokenize import word_tokenize

In [2]:
pattern = r'[' + string.punctuation +']'

In [3]:
#sentences = ("Trong khi đó, trong kỳ họp Quốc hội bất thường khai mạc ngày 4/1, Chính phủ có trình Quốc hội dự thảo một luật sửa 8 luật (trong đó có Luật Đầu tư), nhưng lại không có nội dung về sửa đổi phân cấp, phân quyền chấp thuận chủ trương đầu tư xây dựng khu công nghiệp. 'Chúng tôi đề nghị Chính phủ bổ sung vấn đề này vào để sớm sửa quy định. Đi theo với sự phân cấp, phân quyền là tăng cường kiểm tra, giám sát, cá thể hóa trách nhiệm', ông Quảng đề xuất. Được mời phát biểu, Chủ tịch UBND TP.HCM Phan Văn Mãi cũng nhắc đến một luật sửa 8 luật đang được Chính phủ trình Quốc hội. Ông đề nghị sau khi Quốc hội thông qua thì Chính phủ sớm có hướng dẫn, phân cấp, phân quyền để luật đi vào cuộc sống. Những vấn đề cần phân cấp, phân quyền được Chủ tịch UBND TP.HCM đề xuất là phê chuẩn, điều chỉnh cục bộ quy hoạch, tách dự án giải phóng mặt bằng ra khỏi dự án thông thường... Lãnh đạo TP.HCM cũng đề nghị, kiến nghị sớm sửa đổi Luật Đất đai. Là tham luận cuối cùng trong buổi sáng, Bí thư Thành ủy Thái Nguyên Nguyễn Thanh Hải cũng đề nghị quan tâm phân cấp, phân quyền nhiều hơn cho địa phương. Bà Hải đặc biệt nhấn mạnh đến việc triển khai tách dự án giải phóng mặt bằng ra khỏi dự án thông thường, như vậy mới có thể tạo sự đột biến trong việc thu hút đầu tư.")

In [4]:

def btn_clicked():
    entry1.delete('1.0', END)
    entry1.insert(END, "Loading.....")
    sentences = entry0.get("1.0", "end-1c")
    
    
    print(sentences)
    
    #Number of words in input text
    words_in = ViTokenizer.tokenize(sentences)
    words_in = re.sub(pattern, '', words_in)
    number_of_words_in = len(words_in.split())
    
    sentences=sentences.lower().strip()
    
    sentences = nltk.sent_tokenize(sentences)
    #print(sentences)
    number_of_sentences = len(sentences)
    print("Number_of_sentences: " + str(number_of_sentences))
    
    import sklearn.model_selection
    from gensim.models import KeyedVectors 
    w2v = KeyedVectors.load_word2vec_format("Vectors/vi-300-5-40000-5-fullsplit.vec")
    
    vocab = w2v.vocab
    
    sentences_vec_array = []
    for sentence in sentences:
        sentence = ViTokenizer.tokenize(sentence)
        words = sentence.split(" ")
        sentence_vec = np.zeros((300))
        for word in words:
            if word in vocab:
                sentence_vec+=w2v.wv[word]
        sentences_vec_array.append(sentence_vec)
        
    #write sentences s vectors to file
    #with open('sentence_vec.txt', 'w') as f:
    #f.write(str(sentences_vec_array))
    
    from sklearn.cluster import KMeans

    #n_clusters = 3
    n_clusters = round((number_of_sentences/10)*4)
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans = kmeans.fit(sentences_vec_array)
    
    from sklearn.metrics import pairwise_distances_argmin_min

    avg = []
    for j in range(n_clusters):
        idx = np.where(kmeans.labels_ == j)[0]
        avg.append(np.mean(idx))

    closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, sentences_vec_array)
    ordering = sorted(range(n_clusters), key=lambda k: avg[k])
    summary = ' '.join([sentences[closest[idx]] for idx in ordering])
    
    #Number of words in final text
    words_out = ViTokenizer.tokenize(summary)
    words_out = re.sub(pattern, '', words_out)
    #print(words_out)
    number_of_words_out = len(words_out.split())
    
    #print("tom tat: ")
    #print(summary)
    
    entry1.delete('1.0', END)
    entry1.insert(END, summary)
    
    print("number_of_words_in: " + str(number_of_words_in))
    print("number_of_words_out: " + str(number_of_words_out))
    print("div: " + str((number_of_words_out/number_of_words_in)*100))
    

In [5]:
window = Tk()

window.geometry("1000x573")
window.configure(bg = "#ffffff")
canvas = Canvas(
    window,
    bg = "#ffffff",
    height = 573,
    width = 1000,
    bd = 0,
    highlightthickness = 0,
    relief = "ridge")
canvas.place(x = 0, y = 0)

background_img = PhotoImage(file = f"GUI/background.png")
background = canvas.create_image(
    246.5, 291.0,
    image=background_img)

entry0_img = PhotoImage(file = f"GUI/img_textBox0.png")
entry0_bg = canvas.create_image(
    253.5, 280.5,
    image = entry0_img)

entry0 = Text(
    bd = 0,
    bg = "#ecfeff",
    highlightthickness = 0)

entry0.place(
    x = 35.0, y = 51,
    width = 437.0,
    height = 457)

entry1_img = PhotoImage(file = f"GUI/img_textBox1.png")
entry1_bg = canvas.create_image(
    756.5, 280.5,
    image = entry1_img)

entry1 = Text(
    bd = 0,
    bg = "#cefcff",
    highlightthickness = 0)

entry1.place(
    x = 538.0, y = 51,
    width = 437.0,
    height = 457)

img0 = PhotoImage(file = f"GUI/img0.png")
b0 = Button(
    image = img0,
    borderwidth = 0,
    highlightthickness = 0,
    command = lambda:btn_clicked(),
    relief = "flat")

b0.place(
    x = 200, y = 520,
    width = 108,
    height = 39)

window.resizable(False, False)
window.mainloop()

Theo thống kê giao dịch, thời điểm ông chủ tập đoàn này bán “chui” cổ phiếu, thị giá FLC giao dịch đang ở mức 7.100-7.700 đồng. Như vậy ông Trịnh Văn Quyết có thể đã thu về không dưới 400 tỷ đồng theo giá thị trường từ đợt bán “chui” cổ phiếu này.

Đây cũng là giai đoạn cổ phiếu FLC có thanh khoản cao vượt trội so với bình quân, lên tới hàng chục triệu đơn vị mỗi phiên, đỉnh điểm là phiên 23/10 lên đến 48 triệu đơn vị. Cổ phiếu FLC sau đó cũng rơi nhanh về khoảng 5.700 đồng, tức giảm hơn 20% giá trị so với lúc bán.

Điều đáng nói là số tiền mà đại gia Trịnh Văn Quyết bị xử phạt vì hành vi bán chui 57 triệu cổ phiếu chỉ là 130 triệu đồng.

Trong khi đó ngay sáng nay, Ủy ban Chứng khoán Nhà nước thông báo đã nhận được báo cáo của HoSE đề ngày 10/1 về việc ông Trịnh Văn Quyết đã giao dịch bán 74,8 triệu cổ phiếu FLC mà không báo cáo, không công bố thông tin trước khi thực hiện giao dịch. UBCKNN đang phối hợp với các cơ quan liên quan để xem xét xử lý vi phạm của cá nhân này theo quy định.

C:\Users\lego2\anaconda3\envs\python36-clone-GUI\lib\site-packages\ipykernel_launcher.py:34: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


number_of_words_in: 178
number_of_words_out: 97
div: 54.49438202247191


KeyboardInterrupt: 

# Sentences Tokenizer

# Sentences to vector

In [ ]:

label = kmeans.fit_predict(sentences_vec_array)
print(label)
#sentences_vec_array[1]

In [ ]:
#filter rows of original data
#filtered_label0 = sentences_vec_array[label == 0]


#plotting the results
#plt.scatter(filtered_label0[:,0] , filtered_label0[:,1])
#plt.show()